# 2014年淘宝双12用户行为数据分析


-  项目背景：中国电商行业已经历了十几年的发展，线上购物已成为每个人日常生活中的必备项。各路电商平台厮杀争夺客户，  而电商的人口红利现在已经几乎耗尽，增量时代已经结束。在现如今的存量时代，对于电商平台来说，如何有效对已有用户购   买等行为分析，进而深入了解用户，制定有效的营销运营手段，为实现利润增长变得十分重要。
- 数据收集了淘宝2014年双十二前后（2014年的11月18至12月18）期间的1225万条（12256906）用户购买数据，数据中包含6个字  段，分别是用户id、行为类别（1-4分别代表浏览、收藏、加购、下单）、行为发生时间、地理位置编号、商品id、商品品类。
- 项目目标：对已有用户行为分析，进而深入了解用户，制定有效的营销运营手段，为运营和算法提供建议，实现利润增长
- 分析路径；从人（用户）、货（商品）、场（品平台）三个方面出发对数据进行分析，并针对分析结果，提出综合性的建议

In [1]:
#导入需要的包
import pandas as pd
import numpy as np
import matplotlib

In [2]:
#导入数据并查看数据
sale_data=pd.read_csv('tb_user.csv')
sale_data.head(3)

,user_id,item_id,behavior_type,user_geohash,item_category,time
0,98047837,232431562,1,NaN,4245,2014-12-06 02
1,97726136,383583590,1,NaN,5894,2014-12-09 20
2,98607707,64749712,1,NaN,2883,2014-12-18 11


In [3]:
print(type(sale_data))

<class 'pandas.core.frame.DataFrame'>


In [4]:
sale_data.shape

(12256906, 6)

In [5]:
#数据统计截止时间
sale_data.time.max()

'2014-12-18 23'

In [6]:
#数据统计开始时间
sale_data.time.min()

'2014-11-18 00'

### 1. 数据清洗，主要是重复数据的处理，缺失值的填补、删除，异常数据的处理，以及日期数据的处理

In [7]:
#查看是否有重复数据
sale_data.duplicated().value_counts()

False    8164040
True     4092866
dtype: int64

In [3]:
#数据清洗1：删除重复值
sale_data.drop_duplicates(inplace=True)
sale_data.shape

(8164040, 6)

In [9]:
#数据清洗2：查看缺失情况
sale_data.count()
#此步可以看到，某些用户在使用淘宝时没有开放地理位置权限，导致用户地理位置信息大块缺失，所以此字段暂时不做分析

user_id          8164040
item_id          8164040
behavior_type    8164040
user_geohash     3856025
item_category    8164040
time             8164040
dtype: int64

In [10]:
#数据清洗3：日期格式处理
#time字段是对象类型，需要处理成日期，并得到数据的的年月日，一周中的第几天，一天中的什么时间
sale_data['event_time']=pd.to_datetime(sale_data['time'])
sale_data['event_date'] = sale_data.event_time.dt.date
sale_data['event_hour'] = sale_data.event_time.dt.hour
sale_data['event_weekday'] = sale_data.event_time.dt.dayofweek+1

In [11]:
sale_data['event_time'].head(3)

0   2014-12-06 02:00:00
1   2014-12-09 20:00:00
2   2014-12-18 11:00:00
Name: event_time, dtype: datetime64[ns]

In [12]:
sale_data['event_date'].head(3)

0    2014-12-06
1    2014-12-09
2    2014-12-18
Name: event_date, dtype: object

In [13]:
sale_data['event_hour'].head(3)

0     2
1    20
2    11
Name: event_hour, dtype: int64

In [16]:
sale_data['event_weekday'].head(3)

0    6
1    2
2    4
Name: event_weekday, dtype: int64

In [17]:
#数据清洗4：查看字段数值是否异常
sale_data.describe().applymap(lambda x:'%.2f' % x)

,user_id,item_id,behavior_type,item_category,event_hour,event_weekday
count,8164040.00,8164040.00,8164040.00,8164040.00,8164040.00,8164040.00
mean,71473818.67,202289967.08,1.15,6837.00,15.20,3.92
std,41210851.21,116754363.37,0.54,3805.72,6.18,1.93
min,4913.00,64.00,1.00,2.00,0.00,1.00
25%,35745255.00,101378149.00,1.00,3695.00,11.00,2.00
50%,72703245.00,202059012.50,1.00,6146.00,16.00,4.00
75%,106926411.00,303562703.00,1.00,10275.00,21.00,6.00
max,142455899.00,404562461.00,4.00,14080.00,23.00,7.00


In [18]:
sale_data.describe(include='all')

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,user_id,item_id,behavior_type,user_geohash,item_category,time,event_time,event_date,event_hour,event_weekday
count,8.164040e+06,8.164040e+06,8.164040e+06,3856025,8.164040e+06,8164040,8164040,8164040,8.164040e+06,8.164040e+06
unique,NaN,NaN,NaN,575458,NaN,744,744,31,NaN,NaN
top,NaN,NaN,NaN,94ek6ke,NaN,2014-12-11 22,2014-12-11 22:00:00,2014-12-12,NaN,NaN
freq,NaN,NaN,NaN,1028,NaN,40652,40652,479388,NaN,NaN
first,NaN,NaN,NaN,NaN,NaN,NaN,2014-11-18 00:00:00,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-18 23:00:00,NaN,NaN,NaN
mean,7.147382e+07,2.022900e+08,1.151846e+00,NaN,6.837000e+03,NaN,NaN,NaN,1.520080e+01,3.920420e+00
std,4.121085e+07,1.167544e+08,5.401788e-01,NaN,3.805719e+03,NaN,NaN,NaN,6.177866e+00,1.933076e+00
min,4.913000e+03,6.400000e+01,1.000000e+00,NaN,2.000000e+00,NaN,NaN,NaN,0.000000e+00,1.000000e+00
25%,3.574526e+07,1.013781e+08,1.000000e+00,NaN,3.695000e+03,NaN,NaN,NaN,1.100000e+01,2.000000e+00


In [19]:
sale_data.head(2)

,user_id,item_id,behavior_type,user_geohash,item_category,time,event_time,event_date,event_hour,event_weekday
0,98047837,232431562,1,NaN,4245,2014-12-06 02,2014-12-06 02:00:00,2014-12-06,2,6
1,97726136,383583590,1,NaN,5894,2014-12-09 20,2014-12-09 20:00:00,2014-12-09,20,2


### 2.对数据的总体情况进行分析：

In [ ]:
#用户总体的购物情况
sale_data.behavior_type.count()

可以看到在此期间用户总共发生浏览收藏加购等行为8164040次

In [ ]:
sale_data.groupby('event_date').behavior_type.count().sort_values(ascending=False).head(3)

可以看到，在双12当天用户发生的浏览、收藏、加购、下单行为最多，其次是临近双12的两天。

In [ ]:
sale_data.groupby('event_date').behavior_type.count().sort_values(ascending=True).head(3)

可以看到，在活动开始后的三天，即预热阶段，用户发生的浏览、收藏、加购、下单行为最少，双12当天基本可以达到前三天每天的两倍以上。

In [ ]:
sale_data['user_id'].nunique()

在整个活动期间，使用淘宝的用户人次达到10000

In [ ]:
sale_data.groupby('behavior_type')['user_id'].nunique()

在活动期间吗，使用淘宝下单的人次占活动期间总人次的88.86%，总体来看下单人数占比很大。

In [ ]:
#复购率：产生两次或以上购买次数的用户数占总的购买用户的比率
buy_user=sale_data[sale_data['behavior_type']== 4].groupby('user_id')['behavior_type'].count()
rbuy_user=buy_user[buy_user>=2]
rbuy_rate=round((rbuy_user.count()/8886)*100,2)
print("复购率为"+str(rbuy_rate)+"%")

### 3.用户方向的分析

In [ ]:
#算出用户的购买率（购买数/浏览数）和用户各行为统计
view_count=sale_data[sale_data['behavior_type']== 1].groupby('user_id')['event_time'].count()
buy_count=sale_data[sale_data['behavior_type']== 4].groupby('user_id')['event_time'].count()
col_count=sale_data[sale_data['behavior_type']==2].groupby('user_id')['event_time'].count()
cart_count=sale_data[sale_data['behavior_type']==3].groupby('user_id')['event_time'].count()
buy_rate=buy_count/view_count
df= pd.concat([view_count,col_count,cart_count,buy_count,buy_rate],axis=1)
df.columns=['view_count','col_count','cart_count','buy_count','buy_rate']

In [ ]:
#查看购买率高的用户的行为特征
df.sort_values(by='buy_rate',ascending=False).head(10)

购买率高的客户，浏览量不高，收藏和加购次数也很少，这类客户基本上浏览过后就会直接购买，‘缺啥买啥’属于理智型消费者，广告促销或者推送对这类客户基本不起作用，所以可以考虑减少这类客户的广告推送，或者对这部分客户更精准推送，增加用户好感度和忠诚度

In [ ]:
#查看购买率低的用户的行为特征
df.sort_values(by='buy_rate',ascending=True).head(10)

购买率低的用户基本上基本上可以分为两类，一类是浏览数很高，收藏或者加购次数也多但是购买数却很少，此类客户有可能正在为商家的促销活动做准备，下单欲望不强，购物难度较大；
还有一类是浏览量高，但是收藏和加购次数也不高，这类客户可能对该app的用户粘性不高，只是来对比不同平台商品价格和种类，商品价格或种类不具有吸引力从而流失了用户。

In [ ]:
#用户购买时间分析
%pylab
%matplotlib inline
sale_data.groupby('event_hour')['event_time'].count().plot()

- 从图中可以看出，0点到5点用户活跃次数迅速降低至最低点，6到10点快速上升，10到18点用户活跃数比较平稳，16点后快速上升，21点22点时间段用户使用淘宝次数达到最大，可以根据用户使用时间来投放广告和折扣

### 2. 用户方向的分析

使用RFM模型对用户进行价值分层，从而实现精细化用户运营，数据集不含金额，所以仅从R、F两个维度进行分析   
R——Recency(最近一次购买时间)  
F——Frequency(购买频率)  
M——Money(购物金额)  

In [ ]:
sale_data.reset_index(inplace=True)#由于删除重复数据导致索引混乱，重置索引
sale_data.columns

In [ ]:
#得到R、F值
sale_data_byuid=sale_data[sale_data['behavior_type']==4].groupby(by='user_id')#筛选用户行为为4类的用户，按照用户id分组
RF = pd.DataFrame(index=sale_data_byuid.groups.keys(), columns=['R', 'F'])
RF['F'] = sale_data_byuid.event_time.count()#F为购买行为发生的次数
RF['last_buy_time'] = sale_data_byuid.time.max()#last_buy_time为用户最近一次购买的时间
RF['R']=RF.last_buy_time.apply(lambda x:(pd.to_datetime('2014-12-19 00:00:00')-pd.to_datetime(x)).days)#R为用户最后一次购买距最后记录的天数
RF.drop(['last_buy_time'],axis = 1,inplace=True)#删去辅助行last_buy_time
RF.head(3)

In [ ]:
#得到阈值参考值
RF.describe()

In [ ]:
RFR=RF['R'].tolist()
max(RFR,key=RFR.count)#R的众数

In [ ]:
RFF=RF['F'].tolist()
max(set(RFF),key=RFF.count)#F的众数

- 购买次数平均12次，中位数为8，最少的1次，最多达到770次，而最多的是购买0次的用户，可以看出F的分布是右偏分布，且数据间的差异很大，此时可以用中位数来代表数据是更为合理的，从众数上来看，大部分的用户是仅下载了淘宝APP但并未购买，可能是双12的宣传提高了APP的下载量，但购买转化率太低，这是提高GMV的一个路径
- 购买间隔平均6天，中位数4天，最小0天，最大31天,上四分位数是7天，而最多的是间隔1天就要购买一次的用户，可以看出R是右偏分布，以中位数代表数据，从众数上来看，大部分的用户在双12周期结束前一天都会进行购买，所以可以在最后一天对用户购物车中的商品进行针对性的打折活动等提高GMV

In [ ]:
#一般双12活动的预热在12月10号开始，将距12月18号8天作为划分来判断用户价值,大于8被判断为高价值
RF['R_value']=RF.R.apply(lambda x: 1 if int(x)<8 else 0)
#以消费频次的中位数8次来划分客户价值，大于8为高价值
RF['F_value']=RF.F.apply(lambda x: 1 if int(x)>8 else 0)

In [ ]:
#自定义用户价值判断函数
def user_classfication(tup):
    R_value, F_value = tup
    if R_value == 0 and F_value == 1:#若用户在活动期间12月10号以后没有购买但是总的共购买了8次以上视为重要保持客户
        return "重要保持客户"
    elif R_value == 1 and F_value == 0:#若用户在活动期间12月10号以后有购买但是总的共购买了8次以下视为重要发展客户
        return "重要发展客户"
    elif R_value == 1 and F_value == 1:#若用户在活动期间12月10号以后有购买且总放入共购买了8次以上视为重要价值客户
        return "重要价值客户"
    elif R_value == 0 and F_value == 0:#若用户在活动期间12月10号以后没有购买且总共购买了8次以下视为重要挽留客户
        return "重要挽留客户"
    else:
        return None

In [ ]:
RF['user_classification'] = RF[['R_value','F_value']].apply(user_classfication, axis=1)

In [ ]:
RF.head(5)

In [ ]:
df1=RF.user_classification.value_counts()

In [ ]:
print(df1)

In [ ]:
round(df1.head()*100/RF.user_classification.count(),2)

- 用户部分分析结论：
  重要价值客户 43.90，重要发展客户 32.74，为主要用户群体；
  其中重要价值客户占比43.9%，这类客户不需要进行营销活动，可以为其提供更优质的售后服务，及时解决问题以提高用户的   满意度；
  重要发展客户是32.74%，这类客户是最近有购买，但是最近的购买次数较少的，需要针对其提高消费频次是在重点要关注的客户。


### 3.商品方向的分析

In [8]:
#产品销量分析
sale_data.groupby('item_category').event_time.count().sort_values(ascending=False).head(10)

item_category
1863     264030
13230    231533
5027     225397
5894     217677
6513     196461
5399     191046
11279    123670
2825     108429
5232      96138
10894     92577
Name: event_time, dtype: int64

In [41]:
sale_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8164040 entries, 0 to 8164039
Data columns (total 11 columns):
index            int64
user_id          int64
item_id          int64
behavior_type    int64
user_geohash     object
item_category    int64
time             object
event_time       datetime64[ns]
event_date       object
event_hour       int64
event_weekday    int64
dtypes: datetime64[ns](1), int64(7), object(3)
memory usage: 685.2+ MB


- 对于销量比较靠前的商品，可以用户进行推送或者在首页提高出现的频次，特别是活动商品，也可以通过销量榜单吸引用户购买

In [14]:
#产品关联度分析
#!pip install efficient_apriori
#!pip install mlxtend --user

In [ ]:
"""一、使用efficient_apriori中的apriori方法，但是此方法由于数据集过大占用内存也大，所以很可能跑不出来"""
#获取目标列
from efficient_apriori import apriori
# 统计当天同一客户的订单中购买的所有产品
df_order = sale_data.groupby(['event_date','user_id'])['item_id'].unique()

In [17]:
print(typedf_order)

<class 'pandas.core.series.Series'>


In [ ]:
# 将所有交易订单追加到列表
transactions = []
for value in df_order:
    transactions.append(list(value))

In [148]:
transactions[1:10]

[[164700097, 395281690],
 [389735902,
  343090767,
  396586367,
  205440382,
  223093978,
  321577287,
  286286225,
  76013130,
  331010228,
  76513118,
  158651459,
  366880061],
 [254552786, 216476401, 30759335, 120401989],
 [163031814,
  401255397,
  264424578,
  146548754,
  42478255,
  120198880,
  214110240,
  369276833,
  167249756,
  233751787,
  344486848,
  31115163,
  404162786,
  117663703,
  241305566],
 [371394735,
  382696538,
  282100139,
  248760666,
  327286838,
  123328330,
  392999527,
  54460820,
  322460842,
  349546767,
  384831854,
  158483695,
  88217405,
  376694595,
  93299693,
  397875298,
  345069289,
  388370968,
  230982636,
  354220815,
  379418625,
  237266274,
  372689073,
  191574905,
  169207597,
  381022459,
  316697432],
 [33559828, 70666924, 127932089],
 [219690249, 360748299, 179847493],
 [131168474,
  247726147,
  225759711,
  179104406,
  226140176,
  39873652,
  116768804,
  310869960,
  265634230,
  46876228,
  295465092,
  42321572],
 [27762

In [10]:
# 挖掘频繁项集和频繁规则
import time
start = time.time()
itemsets, rules = apriori(transactions, min_support=0.0001,  min_confidence=0.000001)
print("频繁项集：", itemsets)
print("关联规则：", rules)
end = time.time()
print("用时：",end-start)

频繁项集： {1: {(360820184,): 204, (275450912,): 378, (374235261,): 321, (21087251,): 249, (350186246,): 212, (135104537,): 457, (97655171,): 493, (211781109,): 235, (277922302,): 335, (387911330,): 421, (128186279,): 385, (5685392,): 361, (66399540,): 222, (303205878,): 215, (314903844,): 215, (322554659,): 254, (14087919,): 280, (112921337,): 685, (6703599,): 240, (152338475,): 223, (355292943,): 211, (247894113,): 222, (2217535,): 403, (109421054,): 216, (147989252,): 203, (27364659,): 219, (353381230,): 258, (361681874,): 265, (217213194,): 261, (377464983,): 203, (184181655,): 231, (355491322,): 205, (239702618,): 207, (209323160,): 310}}
关联规则： []
用时： 31.750048398971558


In [13]:
"""二、mlx_apr挖掘频繁项集和频繁规则"""
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [1]:
#由于出现内存溢出问题，所以选择安装较低版本的pandas，获取所需的数据集后再更新到最新版本
#!pip install pandas==0.21
#import pandas as pd
#data_products = sale_data.groupby(sale_data['user_id'])['item_id'].value_counts().unstack()
#!pip install pandas
#data_products[data_products > 1] = 1
#data_products[np.isnan(data_products)] = 0
#此方法耗费时间

In [15]:
#采用分块旋转的方法来处理溢出问题
import pandas as pd 
from tqdm import tqdm

chunk_size = 5000
chunks = [x for x in range(0, sale_data.shape[0], chunk_size)]

for i in range(0, len(chunks) - 1):
    print(chunks[i], chunks[i + 1] - 1)

0 4999
5000 9999
10000 14999
15000 19999
20000 24999
25000 29999
30000 34999
35000 39999
40000 44999
45000 49999
50000 54999
55000 59999
60000 64999
65000 69999
70000 74999
75000 79999
80000 84999
85000 89999
90000 94999
95000 99999
100000 104999
105000 109999
110000 114999
115000 119999
120000 124999
125000 129999
130000 134999
135000 139999
140000 144999
145000 149999
150000 154999
155000 159999
160000 164999
165000 169999
170000 174999
175000 179999
180000 184999
185000 189999
190000 194999
195000 199999
200000 204999
205000 209999
210000 214999
215000 219999
220000 224999
225000 229999
230000 234999
235000 239999
240000 244999
245000 249999
250000 254999
255000 259999
260000 264999
265000 269999
270000 274999
275000 279999
280000 284999
285000 289999
290000 294999
295000 299999
300000 304999
305000 309999
310000 314999
315000 319999
320000 324999
325000 329999
330000 334999
335000 339999
340000 344999
345000 349999
350000 354999
355000 359999
360000 364999
365000 369999
370000 3749

2700000 2704999
2705000 2709999
2710000 2714999
2715000 2719999
2720000 2724999
2725000 2729999
2730000 2734999
2735000 2739999
2740000 2744999
2745000 2749999
2750000 2754999
2755000 2759999
2760000 2764999
2765000 2769999
2770000 2774999
2775000 2779999
2780000 2784999
2785000 2789999
2790000 2794999
2795000 2799999
2800000 2804999
2805000 2809999
2810000 2814999
2815000 2819999
2820000 2824999
2825000 2829999
2830000 2834999
2835000 2839999
2840000 2844999
2845000 2849999
2850000 2854999
2855000 2859999
2860000 2864999
2865000 2869999
2870000 2874999
2875000 2879999
2880000 2884999
2885000 2889999
2890000 2894999
2895000 2899999
2900000 2904999
2905000 2909999
2910000 2914999
2915000 2919999
2920000 2924999
2925000 2929999
2930000 2934999
2935000 2939999
2940000 2944999
2945000 2949999
2950000 2954999
2955000 2959999
2960000 2964999
2965000 2969999
2970000 2974999
2975000 2979999
2980000 2984999
2985000 2989999
2990000 2994999
2995000 2999999
3000000 3004999
3005000 3009999
3010000 

7820000 7824999
7825000 7829999
7830000 7834999
7835000 7839999
7840000 7844999
7845000 7849999
7850000 7854999
7855000 7859999
7860000 7864999
7865000 7869999
7870000 7874999
7875000 7879999
7880000 7884999
7885000 7889999
7890000 7894999
7895000 7899999
7900000 7904999
7905000 7909999
7910000 7914999
7915000 7919999
7920000 7924999
7925000 7929999
7930000 7934999
7935000 7939999
7940000 7944999
7945000 7949999
7950000 7954999
7955000 7959999
7960000 7964999
7965000 7969999
7970000 7974999
7975000 7979999
7980000 7984999
7985000 7989999
7990000 7994999
7995000 7999999
8000000 8004999
8005000 8009999
8010000 8014999
8015000 8019999
8020000 8024999
8025000 8029999
8030000 8034999
8035000 8039999
8040000 8044999
8045000 8049999
8050000 8054999
8055000 8059999
8060000 8064999
8065000 8069999
8070000 8074999
8075000 8079999
8080000 8084999
8085000 8089999
8090000 8094999
8095000 8099999
8100000 8104999
8105000 8109999
8110000 8114999
8115000 8119999
8120000 8124999
8125000 8129999
8130000 

In [ ]:
#运行时间很长且取决于计算机性能，由于数据量过大，很可能跑不出来
pivot_df = pd.DataFrame()
for i in tqdm(range(0, len(chunks) - 1)):
    chunk_df = sale_data.iloc[ chunks[i]:chunks[i + 1] - 1]
    interactions = (chunk_df.groupby('user_id')['item_id']
      .value_counts()
      .unstack()
    )
    pivot_df = pivot_df.append(interactions, sort=False) 
#df_new = pd.concat([df_product.iloc[ chunks[i]:chunks[i + 1] - 1 ].pivot(index='user_id', columns='item_id', values='counts') for i in range(0, len(chunks) - 1)])

  1%|█                                                                            | 23/1632 [06:21<18:21:27, 41.07s/it]

In [100]:
pivot_df[pivot_df> 1] = 1
pivot_df[np.isnan(pivot_df)] = 0

In [ ]:
pivot_df.head(10)

In [2]:
# 挖掘频繁项集
frequent_itemsets = apriori(pivot_df, min_support=0.05, use_colnames=True)
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)

frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
print('频繁项集：', frequent_itemsets)

pd.options.display.max_columns = 100
rules = rules.sort_values(by='lift', ascending=False)
print('关联规则：', rules)

In [ ]:
#使用FP—growth方法 :优点：速度快，缺点：不能挖掘频繁规则
transactions = []
for value in df_order:
    transactions.append(list(str(value)))
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
    def inc(self, numOccur):
        self.count += numOccur
    def disp(self, ind=1):
        print ('  ' * ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind + 1)

# 当出现两个或两个以上的相似项时，找到最后一个相似项的实例，让该实例的self.nodeLink属性保存新出现的相似项
# 效果如同是在一条链的最后一个节点后再接入一个节点，这些链就是self.nodeLink
def updateHeader(nodeToTest, targetNode):
    while nodeToTest.nodeLink != None:
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

# 接收处理好的事务列表，画出FP树
def updateFPtree(items, inTree, headerTable, count):
    if items[0] in inTree.children:
        # 判断items的第一个结点是否已作为子结点
        inTree.children[items[0]].inc(count)
    else:
        # 创建新的分支
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        # 更新相应频繁项集的链表，往后添加
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    # 递归
    if len(items) > 1:
        updateFPtree(items[1::], inTree.children[items[0]], headerTable, count)

# 输入字典格式的事务和最小支持度，返回FP树和项头表
def createFPtree(dataSet, minSup=3):
    headerTable = {}
    for trans in dataSet:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    for k in list(headerTable.keys()):
        if headerTable[k] < minSup:
            del (headerTable[k])  # 删除不满足最小支持度的元素
    freqItemSet = set(headerTable.keys())  # 满足最小支持度的频繁项集
    if len(freqItemSet) == 0:
        return None, None
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]  # element: [count, node]
    retTree = treeNode('Null Set', 1, None)
    for tranSet, count in dataSet.items():
        localD = {}
        for item in tranSet:
            if item in freqItemSet:  # 过滤，只取该样本中满足最小支持度的频繁项
                localD[item] = headerTable[item][0]  # element : count
        if len(localD) > 0:
            # 根据全局频数从大到小对单样本排序
            orderedItem = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            #print('orderItems=', orderedItem)
            # 用过滤且排序后的样本更新树
            updateFPtree(orderedItem, retTree, headerTable, count)
    return retTree, headerTable

# 构造成 element : count 的形式，以字典形式输出
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        trans = "".join(trans).split(',')
        key = frozenset(trans)
        if key in retDict:
            retDict[frozenset(trans)] += 1
        else:
            retDict[frozenset(trans)] = 1
    return retDict

# 递归回溯，找到给定节点往上回溯到根节点的路径，并把路径存到列表中
def ascendFPtree(leafNode, prefixPath):
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendFPtree(leafNode.parent, prefixPath)

# 找到给定元素名称的条件模式基，以字典格式存贮
def findPrefixPath(basePat, myHeaderTab):
    treeNode = myHeaderTab[basePat][1]  # basePat在FP树中的第一个结点
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendFPtree(treeNode, prefixPath)  # prefixPath是倒过来的，从treeNode开始到根
        if len(prefixPath) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count  # 关联treeNode的计数
        treeNode = treeNode.nodeLink  # 下一个basePat结点
    return condPats
def mineFPtree(inTree, headerTable, minSup, preFix, freqItemList):
    # 最开始的频繁项集是headerTable中的各元素
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[1][0])]  # 根据频繁项的总频次排序
    #print("bigL:  ",bigL)
    for basePat in bigL:  # 对每个频繁项
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat, headerTable)  # 当前频繁项集的条件模式基
        myCondTree, myHead = createFPtree(condPattBases, minSup)  # 构造当前频繁项的条件FP树
        if myHead != None:
            mineFPtree(myCondTree, myHead, minSup, newFreqSet, freqItemList)  # 递归挖掘条件FP树              
if __name__ == '__main__':
    parsedDat = transactions
    initSet = createInitSet(parsedDat)
    myFPtree, myHeaderTab = createFPtree(initSet)
    myFPtree.disp()
    myFreqList = []
    mineFPtree(myFPtree, myHeaderTab, 3, set([]), myFreqList)
    print("频繁项集的数量是: %s" % len(myFreqList))
    for item in myFreqList:
        print(item)

### 4.平台分析

- 重要发展用户对比重要价值用户的各个行为之前的转化率，看是哪一步的转化率较低

In [32]:
RF.reset_index(inplace=True)

In [33]:
sale_data_new=pd.merge(sale_data, RF, how='left', left_on='user_id', right_on='index')

In [34]:
developuserrate = sale_data_new[sale_data_new['user_classification'] == '重要发展客户'].groupby(
    'behavior_type').event_time.count()

In [35]:
#重要发展客户计算转化率
df1=pd.DataFrame(developuserrate)
df1.reset_index(inplace=True)
df1.columns=['behavior_type','cnt']
df1['zh_rate1']=df1.cnt*100/df1.cnt.shift(1)
df1['zh_rate2']=df1.cnt*100/df1.cnt.shift(2)

In [39]:
#重要价值客户计算转化率
importantuserrate=sale_data_new[sale_data_new['user_classification']=='重要价值客户'].groupby('behavior_type').event_time.count()
df2=pd.DataFrame(importantuserrate)
df2.reset_index(inplace=True)
df2.columns=['behavior_type','cnt']
df2['zh_rate1']=df2.cnt*100/df2.cnt.shift(1)
df2['zh_rate2']=df2.cnt*100/df2.cnt.shift(2)

In [40]:
#重要发展客户
df1

,behavior_type,cnt,zh_rate1,zh_rate2
0,1,1466974,NaN,NaN
1,2,46065,3.140137,NaN
2,3,44488,96.576577,3.032637
3,4,13503,30.352005,29.312927


In [41]:
#重要价值客户
df2

,behavior_type,cnt,zh_rate1,zh_rate2
0,1,4739371,NaN,NaN
1,2,153470,3.238193,NaN
2,3,246243,160.450251,5.195689
3,4,86811,35.254200,56.565453


- 平台部分分析结论
 重要价值客户中浏览->加购的用户5.2%，重要发展用户为3.0%，低2.2个百分点。
 重要价值客户购物车->下单的比率为35.3%,重要发展客户为30.4%，低3.9个百分点。
 可以考虑从提高重要发展客户浏览到下单，购物车到下单的转化率来提升GMV 。

- 转化率较低的原因：
 浏览至加购转化率低
 1.商品的推荐系统做的不完善有推荐用户需要的商品
 2.活动投放时间未在用户转化率大的时间
 购物车至下单转化率低
 1.用户对商品有意向，但是对价格不满意
 2.忘记自己加购的物品